In [2]:
import pandas as pd
from gurobipy import *
# 此份 Code 為面試展示用，請勿作為其他用途

In [3]:
def restaurant_opt(restaurant,user):
    r_data = pd.read_excel('OR期末data.xlsx', sheet_name = restaurant)    
    u_data = pd.read_excel('OR期末data.xlsx', sheet_name = user)
    place_list = ["無","共同","普通","新生","管一","新體","社科院","男一","女一"]
    week = ["一", "二", "三", "四", "五"]
    H = 158/60
    N = len(r_data["餐廳名稱"])+1
    A = 10
    
    #classroom in the morning
    S = []
    for i in range(31):
        if i%7 == 5 or i%7 == 6:
            S.append([0]*len(place_list))
        else:
            tem = []
            for j in range(len(place_list)):
                tem.append(1 if u_data[week[i%7]][1] == place_list[j] else 0)
            S.append(tem)
        
    #classroom in the afternoon      
    D = []
    for i in range(31):
        if i%7 == 5 or i%7 == 6:
            D.append([0]*len(place_list))
        else:
            tem = []
            for j in range(len(place_list)):
                tem.append(1 if u_data[week[i%7]][2] == place_list[j] else 0)
            D.append(tem)   
        
    #available dining time
    T = []
    for i in range(31):
        if i%7 == 5 or i%7 == 6:
            T.append(0)
        else:
            T.append(300 if u_data[week[i%7]][0] == "不限" else int(u_data[week[i%7]][0]))        

    #average dining time at a restaurant
    E = []
    for j in range(N):
        if j == N-1:
            E.append(0)
        else:
            E.append(float(r_data["平均停留時間 (分鐘)"][j]))
    
    #highest price a customer is willing to pay 
    W = []
    for j in range(N):
        if j == N-1:
            W.append(-1000)
        else:
            W.append(u_data["餐廳偏好"][u_data["餐廳名稱"].tolist().index(r_data["餐廳名稱"][j])])
    
    #budget
    M = int(u_data["一"][4].replace("元/月", ""))

    #restaurant price
    R = []
    for i in range(N):
        if i == N-1:
            R.append(0)
        else:
            R.append(int(r_data["價格區間 (取區間的中位數)"][i]))

    #traveling time
    B = []
    for i in range(N):
        temp = []
        if i == N-1:
            for j in range(len(place_list)):
                temp.append(0)
        else:
            for j in range(len(place_list)):
                temp.append(0 if j == 0 else r_data[place_list[j]][i])
        B.append(temp)

    #restaurant closing days
    O = []
    for i in range(31):
        temp = []
        for j in range(N):
            if j == N-1:
                temp.append(1)
            else:
                a = map(int,str(r_data["公休日(1-7表達，如果都沒有就寫0)"][j]).split(','))
                temp.append(0 if (i%7+1) in a else 1)
        O.append(temp)
        
    #some additional constraints
    C = []
    temp1 = u_data["限制日期"].dropna().tolist()
    temp2 = u_data["限制餐廳"].dropna().tolist()
    if len(temp1) != 0:
        for i in range(len(temp1)):
            temp2[i] = r_data["餐廳名稱"].tolist().index(temp2[i])
            C.append([temp1[i], temp2[i]])
    
# 
    model = Model("mip1")
    #model.setParam('timeLimit', 120)
    model.setParam('MIPGapAbs', 10)
    x = model.addVars(31, N, vtype = GRB.BINARY, name = "x")
    
    model.update()
    model.setObjective(quicksum(quicksum(x[i,j]*(W[j] - R[j]) for i in range(31)) for j in range(N)) +
                   (M - quicksum(quicksum(x[i,j]*R[j] for i in range(31)) for j in range(N))) -
                   quicksum(quicksum(quicksum( (S[i][k] + D[i][k])*B[j][k]*x[i,j]*H
                       for i in range(31)) for j in range(N)) for k in range(len(place_list))) + 
                   quicksum(quicksum(H*(T[i]-E[j])*x[i,j] for i in range(31)) for j in range(N))-
                       quicksum((quicksum(x[i,j]for i in range(31)) - 1) * quicksum(x[i,j]for i in range(31)) 
                                 * A / 2 for j in range(N)), GRB.MAXIMIZE)
    
    #traveling time + dining time <= available time
    for i in range(31):    
        model.addConstr(quicksum(E[j]*x[i,j] for j in range(N)) + quicksum(quicksum(B[j][k]*(S[i][k]+D[i][k])*x[i,j] 
                                                       for k in range(len(place_list))) for j in range(N)) <= T[i])
    #do not exceed budget
    model.addConstr(quicksum(quicksum(x[i,j]*R[j] for i in range(31)) for j in range(N)) <= M)

    #when a restaurant is closed you cannot go there
    for i in range(31):
        for j in range(N):
            model.addConstr(O[i][j] >= x[i,j])
            
    #you can only go to a restaurant twice in seven days
    for n in range(25):
        for j in range(N):
            model.addConstr(quicksum(x[i,j] for i in range(n, n+7)) <= 1)
            
    #you can only eat at one restaurant a day
    for i in range(31):
        if i%7 == 5 or i%7 == 6:
            model.addConstr(quicksum(x[i,j] for j in range(N)) == 0)
        else:
            model.addConstr(quicksum(x[i,j] for j in range(N)) == 1)
            
    #you have specific days to eat at a specific restaurant
    for a in range(len(C)):
        model.addConstr(x[C[a][0] - 1,C[a][1]] == 1)
        
    model.optimize()
    print("Obj:",model.objVal)

    #print results
    for i in range(31):
        for j in range(N):
            if(abs(x[i,j].x)>0.5):
                if j == N-1:
                    print(i+1, "不吃")
                else:
                    
                    print(i+1, r_data["餐廳名稱"][j])

In [3]:
restaurant_opt("餐廳資料","楊鈞智")

Using license file /Users/kzy89426/gurobi.lic
Academic license - for non-commercial use only
Changed value of parameter MIPGapAbs to 10.0
   Prev: 1e-10  Min: 0.0  Max: inf  Default: 1e-10
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4095 rows, 2232 columns and 21466 nonzeros
Model fingerprint: 0x46fcef4a
Model has 35712 quadratic objective terms
Variable types: 0 continuous, 2232 integer (2232 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [3e+00, 2e+03]
  QObjective range [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 3955.6333333
Presolve removed 3027 rows and 996 columns
Presolve time: 0.05s
Presolved: 11477 rows, 11645 columns, 37595 nonzeros
Variable types: 0 continuous, 11645 integer (11645 binary)
Found heuristic solution: objective 7008.3000000

Root relaxation: objective -7.370483e+03, 209 iterations, 0.01 seconds

    Nodes    |    Cur

In [5]:
restaurant_opt("餐廳資料","陳威廷")

Changed value of parameter MIPGapAbs to 10.0
   Prev: 1e-10  Min: 0.0  Max: inf  Default: 1e-10
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4095 rows, 2232 columns and 21466 nonzeros
Model fingerprint: 0x08fc0387
Model has 35712 quadratic objective terms
Variable types: 0 continuous, 2232 integer (2232 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [8e-01, 1e+03]
  QObjective range [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective 3243.1666667
Presolve removed 2854 rows and 765 columns
Presolve time: 0.04s
Presolved: 1241 rows, 1467 columns, 8585 nonzeros
Presolved model has 16401 quadratic objective terms
Variable types: 0 continuous, 1467 integer (1467 binary)

Root relaxation: objective 8.197204e+03, 212 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

In [5]:
restaurant_opt("餐廳資料","郭姿筠")

Changed value of parameter MIPGapAbs to 10.0
   Prev: 1e-10  Min: 0.0  Max: inf  Default: 1e-10
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4096 rows, 2232 columns and 21467 nonzeros
Model fingerprint: 0x42f58290
Model has 35712 quadratic objective terms
Variable types: 0 continuous, 2232 integer (2232 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [8e+00, 1e+03]
  QObjective range [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective 10094.600000
Presolve removed 2824 rows and 707 columns
Presolve time: 0.04s
Presolved: 1272 rows, 1525 columns, 9041 nonzeros
Presolved model has 17040 quadratic objective terms
Variable types: 0 continuous, 1525 integer (1525 binary)

Root relaxation: objective 1.456828e+04, 158 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

In [6]:
restaurant_opt("餐廳資料","陳韋傑")

Changed value of parameter MIPGapAbs to 10.0
   Prev: 1e-10  Min: 0.0  Max: inf  Default: 1e-10
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4095 rows, 2232 columns and 21466 nonzeros
Model fingerprint: 0xc2171614
Model has 35712 quadratic objective terms
Variable types: 0 continuous, 2232 integer (2232 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [3e-01, 1e+03]
  QObjective range [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Found heuristic solution: objective -2046.333333
Presolve removed 3153 rows and 1116 columns
Presolve time: 0.04s
Presolved: 9819 rows, 9993 columns, 32334 nonzeros
Variable types: 0 continuous, 9993 integer (9993 binary)
Found heuristic solution: objective 513.9333333

Root relaxation: objective -7.830667e+02, 571 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

In [7]:
restaurant_opt("餐廳資料","林俊諺")

Changed value of parameter MIPGapAbs to 10.0
   Prev: 1e-10  Min: 0.0  Max: inf  Default: 1e-10
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4095 rows, 2232 columns and 21466 nonzeros
Model fingerprint: 0xf3faa654
Model has 35712 quadratic objective terms
Variable types: 0 continuous, 2232 integer (2232 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [7e-01, 1e+03]
  QObjective range [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective 3666.5000000
Presolve removed 2840 rows and 751 columns
Presolve time: 0.04s
Presolved: 1255 rows, 1481 columns, 8634 nonzeros
Presolved model has 16524 quadratic objective terms
Variable types: 0 continuous, 1481 integer (1481 binary)

Root relaxation: objective 8.288193e+03, 256 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

In [8]:
restaurant_opt("餐廳資料","黃季昕")

Changed value of parameter MIPGapAbs to 10.0
   Prev: 1e-10  Min: 0.0  Max: inf  Default: 1e-10
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4095 rows, 2232 columns and 21466 nonzeros
Model fingerprint: 0x9f062a76
Model has 35712 quadratic objective terms
Variable types: 0 continuous, 2232 integer (2232 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [3e-01, 1e+03]
  QObjective range [1e+01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Found heuristic solution: objective 4350.7666667
Presolve removed 2882 rows and 798 columns
Presolve time: 0.04s
Presolved: 1213 rows, 1434 columns, 8360 nonzeros
Presolved model has 15989 quadratic objective terms
Variable types: 0 continuous, 1434 integer (1434 binary)

Root relaxation: objective 9.130148e+03, 244 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben